In [1]:
# https://www.analyticsvidhya.com/blog/2021/06/vader-for-sentiment-analysis/
# https://github.com/cjhutto/vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tensorflow import keras
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import CountVectorizer
import random
import numpy as np
import string
import nltk
import numpy
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import sklearn
import string
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

def generateSentimentScore(text):
    analyzer = SentimentIntensityAnalyzer()
    
    scores = (analyzer.polarity_scores(text))
    positiveScore = scores['pos'] * 100
    negativeScore = scores['neg'] * 100
    neutralScore = scores['neu'] * 100
    
    if scores['compound'] >= 0.05:
        return ("Positive",positiveScore)
    elif scores['compound'] <= -0.05:
        return ("Negative", negativeScore)
    else:
        return ("Neutral", neutralScore)

In [2]:
model = load_model('chatbot_WIP.h5')

In [3]:
f = open('intents.json', 'r')
responses = json.load(f)
patterns = responses['patterns']
print(responses['patterns'])

[{'tag': '(0,Neutral)', 'responses': ['Hello', 'Hi there']}, {'tag': '(0,Positive)', 'responses': ['Hello!!!!!!']}, {'tag': '(0,Negative)', 'responses': ['Hello :(']}, {'tag': '(1,Neutral)', 'responses': ["You're welcome. Thank you for visiting our website."]}, {'tag': '(1,Positive)', 'responses': ['No problem! Have a great day :)']}, {'tag': '(1,Negative)', 'responses': ['Bye :|']}, {'tag': '(2,Neutral)', 'responses': ['We sell Baseball Gear and baseball related items.']}, {'tag': '(2,Positive)', 'responses': ['We sell baseball gear! Alongside with baseball related items, our store is perfect for you.']}, {'tag': '(2,Negative)', 'responses': ['Baseball stuff. Thats it.']}, {'tag': '(3,Neutral)', 'responses': ['Shipping times are roughly 2 weeks. You can expect a confirmation email with details about your order when you order.']}, {'tag': '(3,Positive)', 'responses': ['Glad you asked. Shipping times are roughly 2 weeks from when you place your order. Your email will have a confirmation

In [4]:
f = open("vocabulary.txt", "r")
vocab = f.read()
vocab = vocab[1:-1].replace('"','')
vocab = vocab.split(",")
print(len(vocab))
vect = CountVectorizer()
vect.fit(vocab)

271


CountVectorizer()

In [5]:
def clean(doc): # doc is a string of text
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([lemmatizer.lemmatize(word.lower()) for word in doc.split()])
    # remove punctuation and numbers
    return doc

def convertToBOW(text):
    text = clean(text)
    text = [text]
    textV = vect.transform(text)
    return textV

In [6]:
model = load_model('chatbot_WIP.h5')

In [7]:
def loadResponse(responseTuple):
    
    predictions, sentiment = responseTuple[0], responseTuple[1]
    
    predictionsIndex = np.argmax((predictions[0]))
    sentimentPNN = sentiment[0]
    
    for jsons in patterns:
        if jsons['tag'] == '(' + str(predictionsIndex) + "," + str(sentimentPNN) + ")":
            return random.choice(jsons['responses'])

In [ ]:
while True:
    print("Enter Text Here: ")
    userInput = input()
    if userInput == "QUIT":
        break
    textVec = convertToBOW(userInput)
    predictions = model.predict(textVec)
    sentimentScore = generateSentimentScore(userInput)
    print(loadResponse((predictions,sentimentScore)))

Enter Text Here: 
Hello
1/1 [==============================] - 0s 262ms/step
Hello
Enter Text Here: 
Hi there! How are you my friend
1/1 [==============================] - 0s 13ms/step
Hello!!!!!!
Enter Text Here: 
I hate this place, but hello
1/1 [==============================] - 0s 11ms/step
Hello :(
Enter Text Here: 
Thank you for your time today
1/1 [==============================] - 0s 12ms/step
No problem! Have a great day :)
Enter Text Here: 
Whatever... thanks I guess?
1/1 [==============================] - 0s 12ms/step
No problem! Have a great day :)
Enter Text Here: 
You're so useless and not helpful, you are not receiving my thanks.
1/1 [==============================] - 0s 10ms/step
Bye :|
Enter Text Here: 
So what do you guys sell here?
1/1 [==============================] - 0s 11ms/step
Hi there
Enter Text Here: 
What items are for sale?
1/1 [==============================] - 0s 9ms/step
Yes Refunds are accepted within 30 days of purchase for an unopened, undamaged item.